In [44]:
import h2o
from h2o.automl import H2OAutoML
import random, os, sys
from datetime import datetime
import pandas as pd
import logging
import csv
import optparse
import time
import json
from distutils.util import strtobool
import psutil

In [45]:
data_path=None
all_variables=None
test_path=None
# target='search_term'
target=None
nthreads=1 
min_mem_size=6 
run_time=200
classification=False
scale=False
max_models=9    
model_path=None
balance_y=False 
balance_threshold=0.2
name=None 
server_path=None  
analysis=0 

In [46]:
pct_memory=0.5
virtual_memory=psutil.virtual_memory()
min_mem_size=int(round(int(pct_memory*virtual_memory.available)/1073741824,0))
print(min_mem_size)

3


In [47]:
# Functions

def alphabet(n):
  alpha='0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'    
  str=''
  r=len(alpha)-1   
  while len(str)<n:
    i=random.randint(0,r)
    str+=alpha[i]   
  return str
  
  
def set_meta_data(analysis,run_id,server,data,test,model_path,target,run_time,classification,scale,model,balance,balance_threshold,name,path,nthreads,min_mem_size):
  m_data={}
  m_data['start_time'] = time.time()
  m_data['target']=target
  m_data['server_path']=server
  m_data['data_path']=data 
  m_data['test_path']=test
  m_data['max_models']=model
  m_data['run_time']=run_time
  m_data['run_id'] =run_id
  m_data['scale']=scale
  m_data['classification']=classification
  m_data['scale']=False
  m_data['model_path']=model_path
  m_data['balance']=balance
  m_data['balance_threshold']=balance_threshold
  m_data['project'] =name
  m_data['end_time'] = time.time()
  m_data['execution_time'] = 0.0
  m_data['run_path'] =path
  m_data['nthreads'] = nthreads
  m_data['min_mem_size'] = min_mem_size
  m_data['analysis'] = analysis
  return m_data


def dict_to_json(dct,n):
  j = json.dumps(dct, indent=4)
  f = open(n, 'w')
  print(j, file=f)
  f.close()
  
  
def stackedensemble(mod):
    coef_norm=None
    try:
      metalearner = h2o.get_model(mod.metalearner()['name'])
      coef_norm=metalearner.coef_norm()
    except:
      pass        
    return coef_norm

def stackedensemble_df(df):
    bm_algo={ 'GBM': None,'GLM': None,'DRF': None,'XRT': None,'Dee': None}
    for index, row in df.iterrows():
      if len(row['model_id'])>3:
        key=row['model_id'][0:3]
        if key in bm_algo:
          if bm_algo[key] is None:
                bm_algo[key]=row['model_id']
    bm=list(bm_algo.values()) 
    bm=list(filter(None.__ne__, bm))             
    return bm

def se_stats(modl):
    d={}
    d['algo']=modl.algo
    d['model_id']=modl.model_id   
    d['auc']=modl.auc()   
    d['roc']=modl.roc()
    d['mse']=modl.mse()   
    d['null_degrees_of_freedom']=modl.null_degrees_of_freedom()
    d['null_deviance']=modl.null_deviance()
    d['residual_degrees_of_freedom']=modl.residual_degrees_of_freedom()   
    d['residual_deviance']=modl.residual_deviance()
    d['rmse']=modl.rmse()
    return d

def get_model_by_algo(algo,models_dict):
    mod=None
    mod_id=None    
    for m in list(models_dict.keys()):
        if m[0:3]==algo:
            mod_id=m
            mod=h2o.get_model(m)      
    return mod,mod_id     
    
    
def gbm_stats(modl):
    d={}
    d['algo']=modl.algo
    d['model_id']=modl.model_id   
    d['varimp']=modl.varimp()  
    return d
    
    
def dl_stats(modl):
    d={}
    d['algo']=modl.algo
    d['model_id']=modl.model_id   
    d['varimp']=modl.varimp()  
    return d
    
    
def drf_stats(modl):
    d={}
    d['algo']=modl.algo
    d['model_id']=modl.model_id   
    d['varimp']=modl.varimp()  
    d['roc']=modl.roc()      
    return d
    
def xrt_stats(modl):
    d={}
    d['algo']=modl.algo
    d['model_id']=modl.model_id   
    d['varimp']=modl.varimp()  
    d['roc']=modl.roc()      
    return d
    
    
def glm_stats(modl):
    d={}
    d['algo']=modl.algo
    d['model_id']=modl.model_id   
    d['coef']=modl.coef()  
    d['coef_norm']=modl.coef_norm()      
    return d
    
def model_performance_stats(perf):
    d={}
    try:    
      d['mse']=perf.mse()
    except:
      pass      
    try:    
      d['rmse']=perf.rmse() 
    except:
      pass      
    try:    
      d['null_degrees_of_freedom']=perf.null_degrees_of_freedom()
    except:
      pass      
    try:    
      d['residual_degrees_of_freedom']=perf.residual_degrees_of_freedom()
    except:
      pass      
    try:    
      d['residual_deviance']=perf.residual_deviance() 
    except:
      pass      
    try:    
      d['null_deviance']=perf.null_deviance() 
    except:
      pass      
    try:    
      d['aic']=perf.aic() 
    except:
      pass      
    try:
      d['logloss']=perf.logloss() 
    except:
      pass    
    try:
      d['auc']=perf.auc()
    except:
      pass  
    try:
      d['gini']=perf.gini()
    except:
      pass    
    return d
    
def impute_missing_values(df, x, scal=False):
    # determine column types
    ints, reals, enums = [], [], []
    for key, val in df.types.items():
        if key in x:
            if val == 'enum':
                enums.append(key)
            elif val == 'int':
                ints.append(key)            
            else: 
                reals.append(key)    
    _ = df[reals].impute(method='mean')
    _ = df[ints].impute(method='median')
    if scal:
        df[reals] = df[reals].scale()
        df[ints] = df[ints].scale()    
    return


def get_independent_variables(df, targ):
    C = [name for name in df.columns if name != targ]
    # determine column types
    ints, reals, enums = [], [], []
    for key, val in df.types.items():
        if key in C:
            if val == 'enum':
                enums.append(key)
            elif val == 'int':
                ints.append(key)            
            else: 
                reals.append(key)    
    x=ints+enums+reals
    return x
    
def get_all_variables_csv(i):
    ivd={}
    try:
      iv = pd.read_csv(i,header=None)
    except:
      sys.exit(1)    
    col=iv.values.tolist()[0]
    dt=iv.values.tolist()[1]
    i=0
    for c in col:
      ivd[c.strip()]=dt[i].strip()
      i+=1        
    return ivd
    
    

def check_all_variables(df,dct,y=None):     
    targ=list(dct.keys())     
    for key, val in df.types.items():
        if key in targ:
          if dct[key] not in ['real','int','enum']:                      
            targ.remove(key)  
    for key, val in df.types.items():
        if key in targ:            
          if dct[key] != val:
            print('convert ',key,' ',dct[key],' ',val)
            if dct[key]=='enum':
                try:
                  df[key] = df[key].asfactor() 
                except:
                  targ.remove(key)                 
            if dct[key]=='int': 
                try:                
                  df[key] = df[key].asnumeric() 
                except:
                  targ.remove(key)                  
            if dct[key]=='real':
                try:                
                  df[key] = df[key].asnumeric()  
                except:
                  targ.remove(key)                  
    if y is None:
      y=df.columns[-1] 
    if y in targ:
      targ.remove(y)
    else:
      y=targ.pop()            
    return targ    
    
def predictions(mod,data,run_id):
    test = h2o.import_file(data)
    mod_perf=mod_best.model_performance(test)
              
    stats_test={}
    stats_test=model_performance_stats(mod_perf)

    n=run_id+'_test_stats.json'
    dict_to_json(stats_test,n) 

    try:    
      cf=mod_perf.confusion_matrix(metrics=["f1","f2","f0point5","accuracy","precision","recall","specificity","absolute_mcc","min_per_class_accuracy","mean_per_class_accuracy"])
      cf_df=cf[0].table.as_data_frame()
      cf_df.to_csv(run_id+'_test_confusion_matrix.csv')
    except:
      pass

    predictions = mod_best.predict(test)
    predictions_df=test.cbind(predictions).as_data_frame() 
    predictions_df.to_csv(run_id+'_predictions.csv')
    return

def predictions_test(mod,test,run_id):
    mod_perf=mod_best.model_performance(test)          
    stats_test={}
    stats_test=model_performance_stats(mod_perf)
    n=run_id+'_test_stats.json'
    dict_to_json(stats_test,n) 
    try:
      cf=mod_perf.confusion_matrix()
#      cf=mod_perf.confusion_matrix(metrics=["f1","f2","f0point5","accuracy","precision","recall","specificity","absolute_mcc","min_per_class_accuracy","mean_per_class_accuracy"])
      cf_df=cf.table.as_data_frame()
      cf_df.to_csv(run_id+'_test_confusion_matrix.csv')
    except:
      pass
    predictions = mod_best.predict(test)    
    predictions_df=test.cbind(predictions).as_data_frame() 
    predictions_df.to_csv(run_id+'_predictions.csv')
    return predictions

def check_X(x,df):
    for name in x:
        if name not in df.columns:
          x.remove(name)  
    return x    
    
    
def get_stacked_ensemble(lst):
    se=None
    for model in model_set:
      if 'BestOfFamily' in model:
        se=model
    if se is None:     
      for model in model_set:
        if 'AllModels'in model:
          se=model           
    return se       
    
def get_variables_types(df):
    d={}
    for key, val in df.types.items():
        d[key]=val           
    return d    
    
#  End Functions

In [48]:
all_variables=None

In [49]:
data_path='C:/Users/prabh/Desktop/ML/Project ML/Dataset/PUBG_data.csv'

In [50]:
run_id=alphabet(9)
if server_path==None:
  server_path=os.path.abspath(os.curdir)
os.chdir(server_path) 
run_dir = os.path.join(server_path,run_id)
os.mkdir(run_dir)
os.chdir(run_dir)    

# run_id to std out
print (run_id) 

H6Q591xpX


In [51]:
logfile=run_id+'_autoh2o_log.zip'
logs_path=os.path.join(run_dir,'logs')
print(logs_path,' ',logfile)

C:\Users\prabh\394zjMEpv\H6Q591xpX\logs   H6Q591xpX_autoh2o_log.zip


In [52]:
# 65535 Highest port no
port_no=random.randint(5555,55555)
h2o.init(strict_version_check=False,min_mem_size_GB=min_mem_size,port=port_no)

Checking whether there is an H2O instance running at http://localhost:40301..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 25.191-b12, mixed mode)
  Starting server from C:\Users\prabh\Anaconda3\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\prabh\AppData\Local\Temp\tmpj4xh7gu3
  JVM stdout: C:\Users\prabh\AppData\Local\Temp\tmpj4xh7gu3\h2o_prabh_started_from_python.out
  JVM stderr: C:\Users\prabh\AppData\Local\Temp\tmpj4xh7gu3\h2o_prabh_started_from_python.err
  Server is running at http://127.0.0.1:40301
Connecting to H2O server at http://127.0.0.1:40301... successful.


H2O cluster uptime:,01 secs
H2O cluster timezone:,America/New_York
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.0.1
H2O cluster version age:,1 month and 17 days
H2O cluster name:,H2O_from_python_prabh_p3dwe4
H2O cluster total nodes:,1
H2O cluster free memory:,3.512 Gb
H2O cluster total cores:,12
H2O cluster allowed cores:,12
H2O cluster status:,"accepting new members, healthy"


In [53]:
# meta data
meta_data = set_meta_data(analysis, run_id,server_path,data_path,test_path,model_path,target,run_time,classification,scale,max_models,balance_y,balance_threshold,name,run_dir,nthreads,min_mem_size)
print(meta_data)  

{'start_time': 1544818907.1292324, 'target': None, 'server_path': 'C:\\Users\\prabh\\394zjMEpv', 'data_path': 'C:/Users/prabh/Desktop/ML/Project ML/Dataset/PUBG_data.csv', 'test_path': None, 'max_models': 9, 'run_time': 200, 'run_id': 'H6Q591xpX', 'scale': False, 'classification': False, 'model_path': None, 'balance': False, 'balance_threshold': 0.2, 'project': None, 'end_time': 1544818907.1292324, 'execution_time': 0.0, 'run_path': 'C:\\Users\\prabh\\394zjMEpv\\H6Q591xpX', 'nthreads': 1, 'min_mem_size': 3, 'analysis': 0}


In [54]:
print(data_path)

C:/Users/prabh/Desktop/ML/Project ML/Dataset/PUBG_data.csv


In [55]:
df = h2o.import_file(data_path)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [56]:
df.head()

Id,groupId,matchId,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,killPoints,kills,killStreaks,longestKill,matchDuration,matchType,maxPlace,numGroups,rankPoints,revives,rideDistance,roadKills,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints
9329eb41e215eb,676b23c24e70d6,45b576ab7daa7f,0,0,51.46,0,0,0,73,0,0,0,0,1884,squad-fpp,28,28,1500,0,0,0,0,0,0,588,1,0
639bd0dcd7bda8,430933124148dd,42a9a0b906c928,0,4,179.1,0,0,2,11,0,2,1,361.9,1811,duo-fpp,48,47,1503,2,4669,0,0,0,0,2017,6,0
63d5c8ef8dfe91,0b45f5db20ba99,87e7e4477a048e,1,0,23.4,0,0,4,49,0,0,0,0,1793,squad-fpp,28,27,1565,0,0,0,0,0,0,787.8,4,0
cf5b81422591d1,b7497dbdc77f4a,1b9a94f1af67f1,0,0,65.52,0,0,0,54,0,0,0,0,1834,duo-fpp,45,44,1465,0,0,0,0,0,0,1812,3,0
ee6a295187ba21,6604ce20a1d230,40754a93016066,0,4,330.2,1,2,1,7,0,3,1,60.06,1326,squad-fpp,28,27,1480,1,0,0,0,0,0,2963,4,0
3e2539b5d78183,029b5a79e08cd6,10186f5c852f62,0,0,0,0,0,0,89,0,0,0,0,1775,squad-fpp,29,29,1490,0,0,0,0,0,0,0,0,0
d812d2f1d88a02,6285bb4eec83e4,f185809740a1a7,0,3,470.7,3,2,17,3,0,5,1,57.61,1328,duo-fpp,49,48,1538,0,0,0,0,0,0,1000,4,0
a8a377e4d43bf8,61ec2e7730a3b8,d31adc82a4930e,0,0,68.61,0,0,0,73,0,0,0,0,1870,squad-fpp,29,27,1487,0,0,0,0,0,0,1217,5,0
f18301e30d47d3,e7d609e08f09b6,bcde504ef16743,0,0,0,0,0,0,56,0,0,0,0,1815,squad-fpp,28,27,1640,0,2355,0,0,0,0,1390,7,0
ec7b965ef978b3,c4c19ef6d6c6d9,6dc5b34c92d5ff,0,0,67.32,0,0,0,54,1023,0,0,0,1336,squad-fpp,27,27,-1,0,0,0,0,0,0,1634,5,1495


In [36]:
df.describe()

Rows:89314
Cols:28




,Id,groupId,matchId,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,killPoints,kills,killStreaks,longestKill,matchDuration,matchType,maxPlace,numGroups,rankPoints,revives,rideDistance,roadKills,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints
type,string,string,enum,int,int,real,int,int,int,int,int,int,int,real,int,enum,int,int,int,int,real,int,real,int,int,real,int,int
mins,NaN,NaN,,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,74.0,,2.0,1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,NaN,NaN,,0.23033343036925902,1.1070604832389102,130.182267484381,0.6576908435407661,0.2265378328145643,1.3597980159885348,47.77985534182771,508.5435765949346,0.9227220816445351,0.5433974516873055,22.899156949638357,1575.8035694292053,,44.78338222451128,43.29070470474954,888.3276977853416,0.15999731285128876,584.7520669100029,0.00335893588911033,4.542568742862261,0.02309828246411537,0.00753521284457084,1148.7277815359298,3.6157713236446694,610.2351479051435
maxs,NaN,NaN,,17.0,19.0,4090.0,31.0,21.0,44.0,100.0,2174.0,41.0,9.0,1004.0,2217.0,,100.0,100.0,3643.0,10.0,40700.0,4.0,1414.0,4.0,4.0,11450.0,141.0,1964.0
sigma,NaN,NaN,,0.5786344123835221,1.7218825612079165,169.84843423112423,1.154834256135213,0.6041728235711351,2.6619435967110805,27.481367317661157,628.6352014718133,1.548634663288867,0.7093500500481391,50.675624308000344,257.164893673424,,23.890016414908757,23.32264730561259,737.5962296007599,0.4584274495301997,1465.0174866221846,0.07010857213394113,30.442279034151706,0.16287600555827525,0.09040301433588624,1181.1052946212326,2.392918316868505,740.4124169930632
zeros,0,0,,73718,50651,24947,55801,74303,53365,0,53102,50791,50791,50963,0,,0,0,1948,77759,66938,89067,83489,87413,88668,2001,4692,53102
missing,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,9329eb41e215eb,676b23c24e70d6,45b576ab7daa7f,0.0,0.0,51.46,0.0,0.0,0.0,73.0,0.0,0.0,0.0,0.0,1884.0,squad-fpp,28.0,28.0,1500.0,0.0,0.0,0.0,0.0,0.0,0.0,588.0,1.0,0.0
1,639bd0dcd7bda8,430933124148dd,42a9a0b906c928,0.0,4.0,179.1,0.0,0.0,2.0,11.0,0.0,2.0,1.0,361.9,1811.0,duo-fpp,48.0,47.0,1503.0,2.0,4669.0,0.0,0.0,0.0,0.0,2017.0,6.0,0.0
2,63d5c8ef8dfe91,0b45f5db20ba99,87e7e4477a048e,1.0,0.0,23.4,0.0,0.0,4.0,49.0,0.0,0.0,0.0,0.0,1793.0,squad-fpp,28.0,27.0,1565.0,0.0,0.0,0.0,0.0,0.0,0.0,787.8,4.0,0.0


In [38]:
'''
import sys
sys.stdout = open('describe.txt', 'w')
print ('test')
'''

"\nimport sys\nsys.stdout = open('describe.txt', 'w')\nprint ('test')\n"

In [58]:
# dependent variable
# assign target and inputs for classification or regression
if target==None:
  target=df.columns[27]   
y = target

In [59]:
print(y)

winPoints


In [60]:
print(all_variables)

None


In [61]:
if all_variables is not None:
  ivd=get_all_variables_csv(all_variables)
  print(ivd)    
  X=check_all_variables(df,ivd,y)
  print(X)

In [62]:
# independent variables

X = []  
if all_variables is None:
  X=get_independent_variables(df, target)  
else: 
  ivd=get_all_variables_csv(all_variables)    
  X=check_all_variables(df, ivd)


X=check_X(X,df)


# Add independent variables

meta_data['X']=X  


# impute missing values

_=impute_missing_values(df,X, scale)

In [63]:
if analysis == 3:
  classification=False
elif analysis == 2:
  classification=True
elif analysis == 1:
  classification=True

In [64]:
# Force target to be factors
# Only 'int' or 'string' are allowed for asfactor(), got Target (Total orders):real 

if classification:
    df[y] = df[y].asfactor()

In [65]:
def check_y(y,df):
  ok=False
  C = [name for name in df.columns if name == y]
  for key, val in df.types.items():
    if key in C:
      if val in ['real','int','enum']:        
        ok=True         
  return ok   

In [66]:
ok=check_y(y,df)
if not ok:
    print(ok)

In [67]:
print(ok)

True


In [68]:
classification=True
if classification:
    print(df[y].levels())

[]


In [69]:
print(df[y].levels())

[]


In [70]:
allV=get_variables_types(df)
allV

{'Id': 'string',
 'groupId': 'string',
 'matchId': 'enum',
 'assists': 'int',
 'boosts': 'int',
 'damageDealt': 'real',
 'DBNOs': 'int',
 'headshotKills': 'int',
 'heals': 'int',
 'killPlace': 'int',
 'killPoints': 'int',
 'kills': 'int',
 'killStreaks': 'int',
 'longestKill': 'real',
 'matchDuration': 'int',
 'matchType': 'enum',
 'maxPlace': 'int',
 'numGroups': 'int',
 'rankPoints': 'int',
 'revives': 'int',
 'rideDistance': 'real',
 'roadKills': 'int',
 'swimDistance': 'real',
 'teamKills': 'int',
 'vehicleDestroys': 'int',
 'walkDistance': 'real',
 'weaponsAcquired': 'int',
 'winPoints': 'int'}

In [71]:
meta_data['variables']=allV

In [72]:
# Set up AutoML

aml = H2OAutoML(max_runtime_secs=run_time,project_name = name)

In [73]:
model_start_time = time.time()

In [74]:
aml.train(x=X,y=y,training_frame=df)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [75]:
meta_data['model_execution_time'] = time.time() - model_start_time

In [76]:
# get leaderboard
aml_leaderboard_df=aml.leaderboard.as_data_frame()

In [77]:
aml_leaderboard_df

,model_id,mean_residual_deviance,rmse,mse,mae,rmsle
0,DRF_1_AutoML_20181214_152402,841.426794,29.007358,841.426794,13.380483,0.079296


In [78]:
# STart best model as first model

model_set=aml_leaderboard_df['model_id']
mod_best=h2o.get_model(model_set[0])

In [79]:
mod_best._id

'DRF_1_AutoML_20181214_152402'

In [80]:
# Get stacked ensemble  
se=get_stacked_ensemble(model_set)

In [81]:
print(se)

None


In [82]:
if se is not None:
  mod_best=h2o.get_model(se)

In [83]:
dir(mod_best)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_bc',
 '_bcin',
 '_check_targets',
 '_compute_algo',
 '_estimator_type',
 '_future',
 '_get_metrics',
 '_have_mojo',
 '_have_pojo',
 '_id',
 '_is_xvalidated',
 '_job',
 '_keyify_if_h2oframe',
 '_make_model',
 '_metrics_class',
 '_model_json',
 '_parms',
 '_plot',
 '_requires_training_frame',
 '_resolve_model',
 '_verify_training_frame_params',
 '_xval_keys',
 'actual_params',
 'aic',
 'algo',
 'auc',
 'balance_classes',
 'biases',
 'binomial_double_trees',
 'build_tree_one_node',
 'calibrate_model',
 'calibration_frame',
 'categorical_encoding',
 'catoffsets',
 'checkpoint',
 'class_sampling_factors',
 'c

In [84]:
mod_best._id

'DRF_1_AutoML_20181214_152402'

In [85]:
mod_best._get_metrics

<function h2o.model.model_base.ModelBase._get_metrics(o, train, valid, xval)>

In [86]:
type(mod_best)

h2o.estimators.random_forest.H2ORandomForestEstimator

In [87]:
mods=mod_best.coef_norm
print(mods)

Model Details
H2ORandomForestEstimator :  Distributed Random Forest
Model Key:  DRF_1_AutoML_20181214_152402


ModelMetricsRegression: drf
** Reported on train data. **

MSE: 870.4707323680038
RMSE: 29.503740989372922
MAE: 13.600397787090085
RMSLE: 0.03553182331689513
Mean Residual Deviance: 870.4707323680038

ModelMetricsRegression: drf
** Reported on validation data. **

MSE: 867.3678951957096
RMSE: 29.45111025404152
MAE: 13.250626391765005
RMSLE: 0.042351829556107765
Mean Residual Deviance: 867.3678951957096

ModelMetricsRegression: drf
** Reported on cross-validation data. **

MSE: 841.4267942744104
RMSE: 29.00735758862586
MAE: 13.38048345893358
RMSLE: 0.07929638796259482
Mean Residual Deviance: 841.4267942744104
Cross-Validation Metrics Summary: 


,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
mae,13.380484,0.0373987,13.377693,13.2955675,13.367249,13.403582,13.458326
mean_residual_deviance,841.42676,5.7397757,842.17426,845.00824,841.0448,827.041,851.8656
mse,841.42676,5.7397757,842.17426,845.00824,841.0448,827.041,851.8656
r2,0.9984657,0.0000105,0.9984702,0.9984561,0.9984625,0.9984918,0.998448
residual_deviance,841.42676,5.7397757,842.17426,845.00824,841.0448,827.041,851.8656
rmse,29.007019,0.0990998,29.020239,29.069025,29.000772,28.758322,29.186737
rmsle,0.0755612,0.0170066,0.0615654,0.0933384,0.0685056,0.1112952,0.0431015


Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance,validation_rmse,validation_mae,validation_deviance
,2018-12-14 15:24:52,50.086 sec,0.0,nan,nan,nan,nan,nan,nan
,2018-12-14 15:24:52,50.415 sec,1.0,39.5114752,17.8549736,1561.1566758,40.7033735,18.0457732,1656.7646173
,2018-12-14 15:24:53,50.637 sec,2.0,38.5968692,17.2216658,1489.7183154,34.7643424,15.7633981,1208.5595048
,2018-12-14 15:24:53,50.862 sec,3.0,37.1694930,16.8417010,1381.5712098,33.0257645,15.0440356,1090.7011209
,2018-12-14 15:24:53,51.047 sec,4.0,36.0850174,16.4718334,1302.1284803,32.1575873,14.5717527,1034.1104196
---,---,---,---,---,---,---,---,---,---
,2018-12-14 15:24:56,53.721 sec,17.0,30.8836468,14.2501408,953.7996385,30.0764311,13.4891044,904.5917093
,2018-12-14 15:24:56,53.920 sec,18.0,30.7963251,14.2155110,948.4136396,30.0229352,13.5165300,901.3766354
,2018-12-14 15:24:56,54.103 sec,19.0,30.6496964,14.1538366,939.4038866,29.9713048,13.4858127,898.2791093
,2018-12-14 15:25:00,58.215 sec,41.0,29.6511909,13.6843138,879.1931246,29.4920015,13.2753575,869.7781552



See the whole table with table.as_data_frame()
Variable Importances: 


variable,relative_importance,scaled_importance,percentage
killPoints,1134802370560.0000000,1.0,0.6357512
rankPoints,638940217344.0000000,0.5630410,0.3579540
matchDuration,6019247616.0000000,0.0053042,0.0033722
numGroups,1040709952.0000000,0.0009171,0.0005830
maxPlace,893386752.0000000,0.0007873,0.0005005
---,---,---,---
swimDistance,24289192.0000000,0.0000214,0.0000136
revives,20898566.0000000,0.0000184,0.0000117
headshotKills,20355848.0000000,0.0000179,0.0000114
vehicleDestroys,2795700.2500000,0.0000025,0.0000016



See the whole table with table.as_data_frame()
<bound method ModelBase.coef_norm of >


In [88]:
'''
mod_best.aic()
 mod_best.algo
 mod_best.coef()
 mod_best.coef_norm()
 mod_best.confusion_matrix()
 mod_best.cross_validation_fold_assignment()
 mod_best.cross_validation_holdout_predictions()
 mod_best.cross_validation_metrics_summary()
 mod_best.cross_validation_models()
 mod_best.cross_validation_predictions()
 mod_best.deepfeatures()
 mod_best.default_params()
 mod_best.download_mojo()
 mod_best.download_pojo()
 mod_best.fit()
 mod_best.full_parameters()
 mod_best.get_params()
 mod_best.get_xval_models()
 mod_best.gini()
 mod_best.have_mojo()
 mod_best.have_pojo()
 mod_best.hit_ratio_table()
 mod_best.is_cross_validated()
 mod_best.join()
 mod_best.keep_levelone_frame()
 mod_best.levelone_frame_id()
 mod_best.logloss()
 mod_best.mae()
 mod_best.mean_per_class_error()
 mod_best.mean_residual_deviance()
 mod_best.metalearner()
 mod_best.metalearner_algorithm()
 mod_best.metalearner_fold_assignment()
 mod_best.metalearner_fold_column()
 mod_best.metalearner_nfolds()
 mod_best.metalearner_params()
 mod_best.mixin()
 mod_best.model_id()
 mod_best.model_performance()
 mod_best.mse()
 mod_best.normmul()
 mod_best.normsub()
 mod_best.null_degrees_of_freedom()
 mod_best.null_deviance()
 mod_best.params()
 mod_best.parms()
 mod_best.partial_plot()
 mod_best.plot()
 mod_best.pprint_coef()
 mod_best.predict()
 mod_best.predict_leaf_node_assignment()
 mod_best.r2()
 mod_best.residual_degrees_of_freedom()
 mod_best.residual_deviance()
 mod_best.respmul()
 mod_best.response_column()
 mod_best.respsub()
 mod_best.rmse()
 mod_best.rmsle()
 mod_best.rotation()
 mod_best.save_model_details()
 mod_best.save_mojo()
 mod_best.score_history()
 mod_best.scoring_history()
 mod_best.seed()
 mod_best.set_params()
 mod_best.show()
 mod_best.start()
 mod_best.std_coef_plot()
 mod_best.summary()
 mod_best.train()
 mod_best.training_frame()
 mod_best.type()
 mod_best.validation_frame()
 mod_best.varimp()
 mod_best.varimp_plot()
 mod_best.weights()
 mod_best.xval_keys()
 mod_best.xvalsmod_best()
 '''

'\nmod_best.aic()\n mod_best.algo\n mod_best.coef()\n mod_best.coef_norm()\n mod_best.confusion_matrix()\n mod_best.cross_validation_fold_assignment()\n mod_best.cross_validation_holdout_predictions()\n mod_best.cross_validation_metrics_summary()\n mod_best.cross_validation_models()\n mod_best.cross_validation_predictions()\n mod_best.deepfeatures()\n mod_best.default_params()\n mod_best.download_mojo()\n mod_best.download_pojo()\n mod_best.fit()\n mod_best.full_parameters()\n mod_best.get_params()\n mod_best.get_xval_models()\n mod_best.gini()\n mod_best.have_mojo()\n mod_best.have_pojo()\n mod_best.hit_ratio_table()\n mod_best.is_cross_validated()\n mod_best.join()\n mod_best.keep_levelone_frame()\n mod_best.levelone_frame_id()\n mod_best.logloss()\n mod_best.mae()\n mod_best.mean_per_class_error()\n mod_best.mean_residual_deviance()\n mod_best.metalearner()\n mod_best.metalearner_algorithm()\n mod_best.metalearner_fold_assignment()\n mod_best.metalearner_fold_column()\n mod_best.met

In [89]:
bm=stackedensemble_df(aml_leaderboard_df)

In [90]:
bm

['DRF_1_AutoML_20181214_152402']

In [91]:
aml_leaderboard_df

,model_id,mean_residual_deviance,rmse,mse,mae,rmsle
0,DRF_1_AutoML_20181214_152402,841.426794,29.007358,841.426794,13.380483,0.079296


In [92]:
#  Get best_models and coef_norm()
best_models={}
best_models=stackedensemble(mod_best)
bm=[]
if best_models is not None: 
  if 'Intercept' in best_models.keys():
    del best_models['Intercept']
  bm=list(best_models.keys())
else:
  best_models={}
  bm=stackedensemble_df(aml_leaderboard_df)   
  for b in bm:   
    best_models[b]=None

if mod_best.model_id not in bm:
    bm.append(mod_best.model_id)

In [93]:
bm

['DRF_1_AutoML_20181214_152402']

In [94]:
# Best of Family leaderboard

aml_leaderboard_df=aml_leaderboard_df.loc[aml_leaderboard_df['model_id'].isin(bm)]

In [95]:
aml_leaderboard_df

,model_id,mean_residual_deviance,rmse,mse,mae,rmsle
0,DRF_1_AutoML_20181214_152402,841.426794,29.007358,841.426794,13.380483,0.079296


In [96]:
# save leaderboard
leaderboard_stats=run_id+'_leaderboard.csv'
aml_leaderboard_df.to_csv(leaderboard_stats)

In [97]:
top=aml_leaderboard_df.iloc[0]['model_id']
print(top)

DRF_1_AutoML_20181214_152402


In [98]:
mod_best=h2o.get_model(top)
print(mod_best._id)
print(mod_best.algo)

DRF_1_AutoML_20181214_152402
drf


In [99]:
meta_data['mod_best']=mod_best._id
meta_data['mod_best_algo']=mod_best.algo

In [100]:
meta_data['models']=bm

In [101]:
models_path=os.path.join(run_dir,'models')
for mod in bm:
  try:   
    m=h2o.get_model(mod) 
    h2o.save_model(m, path = models_path)
  except:    
    pass    

In [102]:
print(models_path)

C:\Users\prabh\394zjMEpv\H6Q591xpX\models


In [104]:
# GBM
 
mod,mod_id=get_model_by_algo("GBM",best_models)
if mod is not None:
    try:     
        sh_df=mod.scoring_history()
        sh_df.to_csv(run_id+'_gbm_scoring_history.csv') 
    except:
        pass   
    try:     
        stats_gbm={}
        stats_gbm=gbm_stats(mod)
        n=run_id+'_gbm_stats.json'
        dict_to_json(stats_gbm,n)
        print(stats_gbm)
    except:
        pass        

In [106]:
if mod is not None:
    try:    
        sh_df=mod.scoring_history()
        sh_df.to_csv(run_id+'_dl_scoring_history.csv') 
    except:
        pass 
    try:
        stats_dl={}
        stats_dl=dl_stats(mod)
        n=run_id+'_dl_stats.json'
        dict_to_json(stats_dl,n)
        print(stats_dl)
    except:
        pass    
    try:
        cf=mod.confusion_matrix()    
        cf_df.to_csv(run_id+'_dl_confusion_matrix.csv')
    except:
        pass       

In [107]:
# DRF

mod,mod_id=get_model_by_algo("DRF",best_models)
if mod is not None:
    try:     
         sh_df=mod.scoring_history()
         sh_df.to_csv(run_id+'_drf_scoring_history.csv') 
    except:
         pass  
    try: 
         stats_drf={}
         stats_drf=drf_stats(mod)
         n=run_id+'_drf_stats.json'
         dict_to_json(stats_drf,n)
         print(stats_drf)
    except:
         pass     

In [108]:
# XRT

mod,mod_id=get_model_by_algo("XRT",best_models)
if mod is not None:
    try:     
         sh_df=mod.scoring_history()
         sh_df.to_csv(run_id+'_xrt_scoring_history.csv')
    except:
         pass     
    try:        
         stats_xrt={}
         stats_xrt=xrt_stats(mod)
         n=run_id+'_xrt_stats.json'
         dict_to_json(stats_xrt,n)
         print(stats_xrt)
    except:
         pass     

In [109]:
# GLM

mod,mod_id=get_model_by_algo("GLM",best_models)
if mod is not None:
    try:     
         stats_glm={}
         stats_glm=glm_stats(mod)
         n=run_id+'_glm_stats.json'
         dict_to_json(stats_glm,n)
         print(stats_glm)
    except:
         pass     

In [110]:
# split into training and test for showing how to predict
train, test = df.split_frame([0.8])

In [111]:
predictions_df=predictions_test(mod_best,test,run_id)

drf prediction progress: |████████████████████████████████████████████████| 100%


In [112]:
predictions_df.head()

predict
0
0
0
1521.18
0
0
0
1495.12
0
1532.18


In [113]:
predictions_df.describe()

Rows:17712
Cols:1




,predict
type,real
mins,0.0
mean,606.1166993403241
maxs,1773.7
sigma,738.8887947016591
zeros,10573
missing,0
0,0.0
1,0.0
2,0.0


In [114]:
# Update and save meta data

meta_data['end_time'] = time.time()
meta_data['execution_time'] = meta_data['end_time'] - meta_data['start_time']
  
n=run_id+'_meta_data.json'
dict_to_json(meta_data,n)

In [115]:
meta_data

{'start_time': 1544818907.1292324,
 'target': None,
 'server_path': 'C:\\Users\\prabh\\394zjMEpv',
 'data_path': 'C:/Users/prabh/Desktop/ML/Project ML/Dataset/PUBG_data.csv',
 'test_path': None,
 'max_models': 9,
 'run_time': 200,
 'run_id': 'H6Q591xpX',
 'scale': False,
 'classification': False,
 'model_path': None,
 'balance': False,
 'balance_threshold': 0.2,
 'project': None,
 'end_time': 1544819914.1848793,
 'execution_time': 1007.0556468963623,
 'run_path': 'C:\\Users\\prabh\\394zjMEpv\\H6Q591xpX',
 'nthreads': 1,
 'min_mem_size': 3,
 'analysis': 0,
 'X': ['assists',
  'boosts',
  'DBNOs',
  'headshotKills',
  'heals',
  'killPlace',
  'killPoints',
  'kills',
  'killStreaks',
  'matchDuration',
  'maxPlace',
  'numGroups',
  'rankPoints',
  'revives',
  'roadKills',
  'teamKills',
  'vehicleDestroys',
  'weaponsAcquired',
  'matchId',
  'matchType',
  'Id',
  'groupId',
  'damageDealt',
  'longestKill',
  'rideDistance',
  'swimDistance',
  'walkDistance'],
 'variables': {'Id': 

In [116]:
# Save logs
h2o.download_all_logs(dirname=logs_path, filename=logfile)

Writing H2O logs to C:\Users\prabh\394zjMEpv\H6Q591xpX\logs\H6Q591xpX_autoh2o_log.zip


'C:\\Users\\prabh\\394zjMEpv\\H6Q591xpX\\logs\\H6Q591xpX_autoh2o_log.zip'

In [117]:
# Clean up
os.chdir(server_path)

In [118]:
def predictions_test(mod,test,run_id):
    mod_perf=mod_best.model_performance(test)          
    stats_test={}
    stats_test=model_performance_stats(mod_perf)
    print(stats_test)   
    n=run_id+'_test_stats.json'
    dict_to_json(stats_test,n) 
    try:
      cf=mod_perf.confusion_matrix()
      print(cf)   
#      cf=mod_perf.confusion_matrix(metrics=["f1","f2","f0point5","accuracy","precision","recall","specificity","absolute_mcc","min_per_class_accuracy","mean_per_class_accuracy"])
      cf_df=cf.table.as_data_frame()
      print(cf_df)    
      cf_df.to_csv(run_id+'_test_confusion_matrix.csv')
    except:
      pass
    predictions = mod_best.predict(test)    
    predictions_df=test.cbind(predictions).as_data_frame() 
    predictions_df.to_csv(run_id+'_predictions.csv')
    return predictions

In [119]:
h2o.cluster().shutdown()

H2O session _sid_891b closed.
